In [1]:
import pandas as pd
import numpy as np
dir_dba = r'E:\MEGA\PMMG\MySql\10 - Tbl_dimensao\pdca\tbl_base_PDCA_2020.xls'#Home
#dir_dba = r'C:\Users\Geo\Documents\MEGAsync\MySql\10 - Tbl_dimensao\pdca\tbl_base_PDCA_2020.xls'#Office
df_ocorrencias = pd.read_excel(dir_dba, sheet_name='tbl_ocorrencias')
df_armas_fgo = pd.read_excel(dir_dba, sheet_name='tbl_armas_fgo')
df_envolvidos = pd.read_excel(dir_dba, sheet_name='tbl_envolvidos')
df_veiculos = pd.read_excel(dir_dba, sheet_name='tbl_veiculos')
df_materiais = pd.read_excel(dir_dba, sheet_name='tbl_materiais')
df_infracoes = pd.read_excel(dir_dba, sheet_name='tbl_infracoes')
df_integrantes = pd.read_excel(dir_dba, sheet_name='tbl_integrantes')
print("Arquivos Lidos com sucesso!!!...")

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Arquivos Lidos com sucesso!!!...


In [2]:
for df in [df_ocorrencias,df_armas_fgo,df_envolvidos,df_veiculos,df_materiais,df_infracoes,df_integrantes]:
    df.columns = df.columns.str.replace(' ', '_')
    df.columns = df.columns.str.upper()
df_envolvidos.columns   

Index(['NÚMERO_REDS', 'ÓRGÃO_UNIDADE_REGISTRO', 'UNIDADE_ÁREA_MILITAR',
       'UNID_REGISTRO_NÍVEL_6', 'CÓDIGO_SUBCLASSE_NAT_PRINCIPAL',
       'TENTADO/CONSUMADO_NAT_PRINCIPAL', 'CÓDIGO_SUBCLASSE_NATUREZA',
       'TENTADO/CONSUMADO', 'GRAU_LESÃO', 'TIPO_ENVOLVIMENTO',
       'GRUPO_TIPO_ENVOLVIMENTO', 'PRISÃO_/_APREENSÃO', 'DATA_FATO',
       'HORÁRIO_FATO', 'DATA_ÚLTIMA_ATUALIZAÇÃO',
       'LOGRADOURO_OCORRÊNCIA_-_TIPO', 'LOGRADOURO_OCORRÊNCIA',
       'LOGRADOURO_OCORRÊNCIA_NÃO_CADASTRADO', 'LOGRADOURO_CRUZAMENTO',
       'LOGRADOURO_CRUZAMENTO_NÃO_CADASTRADO', 'NÚMERO_LOGRADOURO', 'BAIRRO',
       'BAIRRO_NÃO_CADASTRADO', 'COMPLEMENTO_ENDEREÇO', 'PONTO_DE_REFERÊNCIA',
       'MUNICÍPIO', 'LATITUDE', 'LONGITUDE', 'NOME_ENVOLVIDO',
       'DATA_NASCIMENTO', 'SEXO', 'CÚTIS', 'IDADE_APARENTE', 'ESTADO_CIVIL',
       'OCUPAÇÃO_ATUAL', 'ESCOLARIDADE', 'LOGRADOURO_ENVOLVIDO',
       'BAIRRO_ENVOLVIDO', 'TIPO_LOGRADOURO_ENVOLVIDO', 'MUNICÍPIO_ENVOLVIDO',
       'QTDE_ENVOLVIDOS'],
     

In [3]:
df_env = df_envolvidos.groupby(['NÚMERO_REDS']).size().to_frame('ENVOLVIDOS').reset_index()

#df_env['ENV'] = df_envolvidos.groupby('NÚMERO_REDS')['NÚMERO_REDS'].transform('count')

#envolvidos = df_envolvidos.groupby('NÚMERO_REDS')["NÚMERO_REDS"].count().rename("ENVOLVIDOS").reset_index()
#df_env = df_env.merge(envolvidos)

presos = df_envolvidos[df_envolvidos['PRISÃO_/_APREENSÃO']=='FLAGRANTE DE CRIME / CONTRAVENCAO']\
                            .groupby('NÚMERO_REDS')["NÚMERO_REDS"].count().rename("PRESOS").reset_index()
df_env = df_env.merge(presos, how="outer")

apreendidos = df_envolvidos[df_envolvidos['PRISÃO_/_APREENSÃO']=='FLAGRANTE DE ATO INFRACIONAL']\
                            .groupby('NÚMERO_REDS')["NÚMERO_REDS"].count().rename("APREENDIDOS").reset_index()
df_env = df_env.merge(apreendidos, how="outer")

presos_roubo = df_envolvidos[(df_envolvidos['PRISÃO_/_APREENSÃO']=='FLAGRANTE DE CRIME / CONTRAVENCAO')\
                             & (df_envolvidos['CÓDIGO_SUBCLASSE_NATUREZA'] == 'C01157')]\
                            .groupby('NÚMERO_REDS')["NÚMERO_REDS"].count().rename("PRESOS_ROUBO").reset_index()
df_env = df_env.merge(presos_roubo, how="outer")



df_env.tail()



save = r"C:\Users\Luísa\Desktop\df_env.xlsx"
df_env.to_excel(save, sheet_name='validados', index = False)


In [4]:
nats = ('C01157')
prisao_apreensao = ('FLAGRANTE DE CRIME / CONTRAVENCAO')
is_tcaf = [
            (df_envolvidos ['CÓDIGO_SUBCLASSE_NAT_PRINCIPAL']==(nats)) &
            (df_envolvidos ['PRISÃO_/_APREENSÃO']==(prisao_apreensao))
           ]

df_envolvidos['PRESOS_ROUBO'] = np.select(is_tcaf, [1], default=0)

df_envolvidos


,NÚMERO_REDS,ÓRGÃO_UNIDADE_REGISTRO,UNIDADE_ÁREA_MILITAR,UNID_REGISTRO_NÍVEL_6,CÓDIGO_SUBCLASSE_NAT_PRINCIPAL,TENTADO/CONSUMADO_NAT_PRINCIPAL,CÓDIGO_SUBCLASSE_NATUREZA,TENTADO/CONSUMADO,GRAU_LESÃO,TIPO_ENVOLVIMENTO,...,IDADE_APARENTE,ESTADO_CIVIL,OCUPAÇÃO_ATUAL,ESCOLARIDADE,LOGRADOURO_ENVOLVIDO,BAIRRO_ENVOLVIDO,TIPO_LOGRADOURO_ENVOLVIDO,MUNICÍPIO_ENVOLVIDO,QTDE_ENVOLVIDOS,PRESOS_ROUBO
0,2020-005344138-001,POLICIA MILITAR,2 GP/3 PEL/51 CIA PM/23 BPM/7 RPM,51 CIA PM/23 BPM/7 RPM,C01155,CONSUMADO,C01155,CONSUMADO,SEM LESOES APARENTES,VITIMA DE ACAO CRIMINAL / CIVEL,...,72.0,CASADO,APOSENTADO,ENSINO FUNDAMENTAL INCOMPLETO (8 ANOS ESTUDO),ANTONIO MARCELO COURA,PARQUE SAO JOSE,RUA,BELO HORIZONTE,1,0
1,2020-015129934-002,POLICIA MILITAR,2 PEL/141 CIA PM/7 BPM/7 RPM,141 CIA PM/7 BPM/7 RPM,A99000,CONSUMADO,A99000,CONSUMADO,PREENCHIMENTO OPCIONAL,SOLICITANTE,...,41.0,ESTADO CIVIL - IGNORADO,NaN,SUPERIOR COMPLETO,DA SAUDADE,CENTRO,AVENIDA,DORES DO INDAIA,1,0
2,2020-016271506-001,POLICIA MILITAR,3 PEL/19 CIA PM IND/7 RPM,19 CIA PM IND/7 RPM,T10161,CONSUMADO,T10161,CONSUMADO,SEM LESOES APARENTES,CONDUTOR DO VEICULO,...,23.0,ESTADO CIVIL - NAO DECLARADO,NaN,ALFABETIZADO,NOSSA SENHORA APARECIDA,VILA NOSSA SENHORA APARECIDA,RUA,PARA DE MINAS,1,0
3,2020-019637565-001,POLICIA MILITAR,142 CIA PM/23 BPM/7 RPM,142 CIA PM/23 BPM/7 RPM,B01147,CONSUMADO,B01147,CONSUMADO,LEVES,AUTOR,...,41.0,CASADO,PEDREIRO,ESCOLARIDADE - IGNORADA,NAO INFORMADO,NAO INFORMADO,NAO INFORMADO,SEM INFORMAÇÃO,1,0
4,2020-019637565-001,POLICIA MILITAR,142 CIA PM/23 BPM/7 RPM,142 CIA PM/23 BPM/7 RPM,B01147,CONSUMADO,B01147,CONSUMADO,SEM LESOES APARENTES,VITIMA DE ACAO CRIMINAL / CIVEL,...,37.0,UNIAO ESTAVEL,COSTUREIRA,ESCOLARIDADE - IGNORADA,JOAQUIM MANOEL PEREIRA,SAGRADA FAMILIA,RUA,DIVINOPOLIS,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12056,2020-036441790-001,POLICIA MILITAR,142 CIA PM/23 BPM/7 RPM,142 CIA PM/23 BPM/7 RPM,G02024,CONSUMADO,G02024,CONSUMADO,SEM LESOES APARENTES,AUTOR,...,46.0,CASADO,NaN,ANALFABETO,ITARARE,ICARAI,RUA,DIVINOPOLIS,1,0
12057,2020-036441790-001,POLICIA MILITAR,142 CIA PM/23 BPM/7 RPM,142 CIA PM/23 BPM/7 RPM,G02024,CONSUMADO,G02024,CONSUMADO,SEM LESOES APARENTES,VITIMA DE ACAO CRIMINAL / CIVEL,...,42.0,CASADO,NaN,ENSINO FUNDAMENTAL COMPLETO (8 ANOS ESTUDO),TENECI,JARDIM FLORAMAR,AVENIDA,DIVINOPOLIS,1,0
12058,2020-036442899-001,POLICIA MILITAR,142 CIA PM/23 BPM/7 RPM,142 CIA PM/23 BPM/7 RPM,A08000,CONSUMADO,A08000,CONSUMADO,SEM LESOES APARENTES,AUTOR,...,29.0,ESTADO CIVIL - IGNORADO,NaN,ALFABETIZADO,TULIPAS,DEL REY,RUA,DIVINOPOLIS,1,0
12059,2020-036445379-001,POLICIA MILITAR,3 PEL/142 CIA PM/23 BPM/7 RPM,7 CIA PM RV/BPMRV/CPRV,I04028,CONSUMADO,I04028,CONSUMADO,SEM LESOES APARENTES,AUTOR,...,18.0,SOLTEIRO,NaN,ALFABETIZADO,AMG 0330,NAO INFORMADO,RODOVIA,CARMO DO CAJURU,1,0


In [5]:
prisao_apreensao = ('FLAGRANTE DE ATO INFRACIONAL', 'FLAGRANTE DE CRIME / CONTRAVENCAO','MANDADO JUDICIAL','OUTRAS - PRISAO / APREENSAO', 'RECAPTURA')

is_tcaf = [
            (df_envolvidos ['PRISÃO_/_APREENSÃO'].isin(prisao_apreensao))
            ]

df_envolvidos['PRESOS_GERAL'] = np.select(is_tcaf, [1], default=0)



In [6]:
df_envolvidos

,NÚMERO_REDS,ÓRGÃO_UNIDADE_REGISTRO,UNIDADE_ÁREA_MILITAR,UNID_REGISTRO_NÍVEL_6,CÓDIGO_SUBCLASSE_NAT_PRINCIPAL,TENTADO/CONSUMADO_NAT_PRINCIPAL,CÓDIGO_SUBCLASSE_NATUREZA,TENTADO/CONSUMADO,GRAU_LESÃO,TIPO_ENVOLVIMENTO,...,ESTADO_CIVIL,OCUPAÇÃO_ATUAL,ESCOLARIDADE,LOGRADOURO_ENVOLVIDO,BAIRRO_ENVOLVIDO,TIPO_LOGRADOURO_ENVOLVIDO,MUNICÍPIO_ENVOLVIDO,QTDE_ENVOLVIDOS,PRESOS_ROUBO,PRESOS_GERAL
0,2020-005344138-001,POLICIA MILITAR,2 GP/3 PEL/51 CIA PM/23 BPM/7 RPM,51 CIA PM/23 BPM/7 RPM,C01155,CONSUMADO,C01155,CONSUMADO,SEM LESOES APARENTES,VITIMA DE ACAO CRIMINAL / CIVEL,...,CASADO,APOSENTADO,ENSINO FUNDAMENTAL INCOMPLETO (8 ANOS ESTUDO),ANTONIO MARCELO COURA,PARQUE SAO JOSE,RUA,BELO HORIZONTE,1,0,0
1,2020-015129934-002,POLICIA MILITAR,2 PEL/141 CIA PM/7 BPM/7 RPM,141 CIA PM/7 BPM/7 RPM,A99000,CONSUMADO,A99000,CONSUMADO,PREENCHIMENTO OPCIONAL,SOLICITANTE,...,ESTADO CIVIL - IGNORADO,NaN,SUPERIOR COMPLETO,DA SAUDADE,CENTRO,AVENIDA,DORES DO INDAIA,1,0,0
2,2020-016271506-001,POLICIA MILITAR,3 PEL/19 CIA PM IND/7 RPM,19 CIA PM IND/7 RPM,T10161,CONSUMADO,T10161,CONSUMADO,SEM LESOES APARENTES,CONDUTOR DO VEICULO,...,ESTADO CIVIL - NAO DECLARADO,NaN,ALFABETIZADO,NOSSA SENHORA APARECIDA,VILA NOSSA SENHORA APARECIDA,RUA,PARA DE MINAS,1,0,0
3,2020-019637565-001,POLICIA MILITAR,142 CIA PM/23 BPM/7 RPM,142 CIA PM/23 BPM/7 RPM,B01147,CONSUMADO,B01147,CONSUMADO,LEVES,AUTOR,...,CASADO,PEDREIRO,ESCOLARIDADE - IGNORADA,NAO INFORMADO,NAO INFORMADO,NAO INFORMADO,SEM INFORMAÇÃO,1,0,1
4,2020-019637565-001,POLICIA MILITAR,142 CIA PM/23 BPM/7 RPM,142 CIA PM/23 BPM/7 RPM,B01147,CONSUMADO,B01147,CONSUMADO,SEM LESOES APARENTES,VITIMA DE ACAO CRIMINAL / CIVEL,...,UNIAO ESTAVEL,COSTUREIRA,ESCOLARIDADE - IGNORADA,JOAQUIM MANOEL PEREIRA,SAGRADA FAMILIA,RUA,DIVINOPOLIS,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12056,2020-036441790-001,POLICIA MILITAR,142 CIA PM/23 BPM/7 RPM,142 CIA PM/23 BPM/7 RPM,G02024,CONSUMADO,G02024,CONSUMADO,SEM LESOES APARENTES,AUTOR,...,CASADO,NaN,ANALFABETO,ITARARE,ICARAI,RUA,DIVINOPOLIS,1,0,1
12057,2020-036441790-001,POLICIA MILITAR,142 CIA PM/23 BPM/7 RPM,142 CIA PM/23 BPM/7 RPM,G02024,CONSUMADO,G02024,CONSUMADO,SEM LESOES APARENTES,VITIMA DE ACAO CRIMINAL / CIVEL,...,CASADO,NaN,ENSINO FUNDAMENTAL COMPLETO (8 ANOS ESTUDO),TENECI,JARDIM FLORAMAR,AVENIDA,DIVINOPOLIS,1,0,0
12058,2020-036442899-001,POLICIA MILITAR,142 CIA PM/23 BPM/7 RPM,142 CIA PM/23 BPM/7 RPM,A08000,CONSUMADO,A08000,CONSUMADO,SEM LESOES APARENTES,AUTOR,...,ESTADO CIVIL - IGNORADO,NaN,ALFABETIZADO,TULIPAS,DEL REY,RUA,DIVINOPOLIS,1,0,1
12059,2020-036445379-001,POLICIA MILITAR,3 PEL/142 CIA PM/23 BPM/7 RPM,7 CIA PM RV/BPMRV/CPRV,I04028,CONSUMADO,I04028,CONSUMADO,SEM LESOES APARENTES,AUTOR,...,SOLTEIRO,NaN,ALFABETIZADO,AMG 0330,NAO INFORMADO,RODOVIA,CARMO DO CAJURU,1,0,1


In [9]:
pd.pivot_table(df_envolvidos,
               index=['NÚMERO_REDS'], 
               values=['QTDE_ENVOLVIDOS', 'PRESOS_GERAL', 'PRESOS_ROUBO'], 
               fill_value=0,
               aggfunc=np.sum, 
               #margins=True, 
               #margins_name='Total Geral'
               
              ).reset_index()

,NÚMERO_REDS,PRESOS_GERAL,PRESOS_ROUBO,QTDE_ENVOLVIDOS
0,2020-005344138-001,0,0,1
1,2020-015129934-002,0,0,1
2,2020-016271506-001,0,0,1
3,2020-019637565-001,1,0,3
4,2020-026146770-002,0,0,2
...,...,...,...,...
6541,2020-036436600-001,2,0,4
6542,2020-036441790-001,1,0,4
6543,2020-036442899-001,1,0,1
6544,2020-036445379-001,1,0,1
